# my-project-001

Use the "Run" button to execute the code.

In [ ]:
!pip install jovian --upgrade --quiet

In [ ]:
import jovian

In [ ]:
# Execute this to save new versions of the notebook
jovian.commit(project="my-project-001")

## Driver Alertness Prediction
In this project we will try to predict the Alertness of the driver based on given data.

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

In [ ]:
od.version()

In [ ]:
dataset_url = 'https://www.kaggle.com/c/stayalert'

In [ ]:
od.download(dataset_url)

In [ ]:
import os

In [ ]:
data_dir = './stayalert'

In [ ]:
os.listdir(data_dir)

In [ ]:
train_csv = data_dir + '/fordTrain.csv'

In [ ]:
import pandas as pd

In [ ]:
take1 = pd.read_csv(train_csv)

In [ ]:
test1 = pd.read_csv(data_dir + '/fordTest.csv')

In [ ]:
take1

In [ ]:
take1.info()

In [ ]:
sum(take1['IsAlert']==1)/604329

### If we just take all the IsAlert Answers to be 1, We will get the Accuracy of 57%.

In [ ]:
take1.loc[:,"V9"]

In [ ]:
sum(take1['V9']==1) # every id in V9 is given 0

### There are multiple rows with all entries as 0, So we will delete that entries.

In [ ]:
!pip install matplotlib plotly seaborn --upgrade --quiet

In [ ]:
import plotly as py

In [ ]:

import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['figure.figsize']=(10,6)
matplotlib.rcParams['figure.facecolor']='#00000000'

In [ ]:
px.histogram(take1,
            x = 'P1',
            title = 'ANALYSIS_101',
            color = 'IsAlert')


In [ ]:
px.histogram(take1,
            x = 'P6',
            title = 'ANALYSIS_101',
            color = 'IsAlert')

In [ ]:
px.scatter(take1,
            x = 'V5',
            title = 'ANALYSIS_101',
            y = 'IsAlert')

All V9 values are 0 so we will have to take this into account. There are many colums that are totally Zero. We can just take them out of our dataset.

In [ ]:
take1.describe()

In [ ]:
sum(take1['P8']==0)

In [ ]:
sum(take1['V7']==0)

In [ ]:
sum(take1['V9']==0)

Seeing that these three colums have output 0 for every input it will be nice to delete the columns altogether.

In [ ]:
take1 = take1.drop(['P8','V7','V9'],axis = 1)

In [ ]:
take1.describe().T

Let us check the correlations between the different columns.

In [ ]:
fig = px.imshow(take1.corr())
fig.show()

In [ ]:
px.scatter(take1,
            x = 'P3',
            y = 'P4',
            color = 'IsAlert')

-  There is negative correlation between P3 and P4.
-  Positive correlation between V1,V6 and V1,V10.
Let us see the graphs.

---



In [ ]:
px.scatter(take1,
           x = 'V1',
           y = 'V10',
           color = 'IsAlert')

In [ ]:
jovian.commit(project = 'my-project-001')

- Now we will normalize and try to fit in logistic regression.
- Cleaning was required just to take out the irrelevant columns.
- No null values were found.
- We can not decide how to change the different columns as we dont know what they represent.
- We just have to use less and less columns of physiological data.
- So lets try to get accuracy with diferent models. 
- Remember the process is a loop so we can come back if we dont feel content.

In [ ]:
!pip install scikit-learn --upgrade --quiet

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, val_df = train_test_split(take1, test_size = 0.25, random_state = 42)

In [ ]:
train_df

In [ ]:
X =take1.iloc[:,3:31]

In [ ]:
X

In [ ]:
X_train,X_val = train_test_split(train_df, test_size = 0.25, random_state = 42)

LET US Normalize.
### We have taken the train part and the validation part seperately, and also we have taken the targets out so that we will now start the show called ML.

In [ ]:
X_train_target = X_train.iloc[:,2]
X_train_target

In [ ]:
X_val_target =X_val.iloc[:,2]

In [ ]:
X_train_input = X_train.iloc[:,3:31]
X_val_input = X_val.iloc[:,3:31]
X_train_input

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
input_X=train_df.iloc[:,3:31]

In [ ]:
input_X

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(input_X)

In [ ]:
list(scaler.data_min_)

In [ ]:
X_train_input

In [ ]:
X_train_input.iloc[:,:] = scaler.transform(X_train_input.iloc[:,:])
X_val_input.iloc[:,:] = scaler.transform(X_val_input.iloc[:,:])

In [ ]:
X_train_input

### Now we have normalised the data and now we can start with the different models. The targets are either 0 or 1. We may try to do logistic regression but we are not.

In [ ]:
jovian.commit()

In [ ]:
jovian.commit(project="my-project-001")

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model=DecisionTreeRegressor(random_state = 42)

In [ ]:
model.fit(X_train_input, X_train_target)

In [ ]:
preds_tree =model.predict(X_train_input)
preds_tree

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(X_train_target, preds_tree)

This is more than perfect...
I think it has mugged up the answers. We will have to tweak with the hyperparameters of the model, untill then let us see what it comments on validation set.

In [ ]:
jovian.commit(project = 'my-project-001')

In [ ]:
X_train_target

In [ ]:
pred_val_tree = model.predict(X_val_input)

In [ ]:
accuracy_score(X_val_target,pred_val_tree)

In [ ]:
jovian.commit(project = 'my-project-001')

### Now we will have a look at random forest. Let us have a random forest evaluate it.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
jungle = RandomForestRegressor(random_state = 42)

In [ ]:
jungle.fit(X_train_input,X_train_target)

In [ ]:
jungle_preds = jungle.predict(X_train_input)

In [ ]:
def tologistic(jungle_preds):
    for i in range(len(jungle_preds)):
        if jungle_preds[i]>0.5:
            jungle_preds[i]=1
        else:
            jungle_preds[i]=0
    return jungle_preds

In [ ]:
tologistic(jungle_preds)

In [ ]:
accuracy_score(X_train_target,jungle_preds )

In [ ]:
jungle_val_preds =jungle.predict(X_val_input)

In [ ]:
tologistic(jungle_val_preds)

In [ ]:
accuracy_score(X_val_target, jungle_val_preds)

### So we will try tuning the hyperparameters, What do you say or validation is good enough.
Should we try a test set one time, Oh I remembered how can we test the test??

In [ ]:
jovian.commit()

## Let us tune the model a little to get more realistic answers or should I get the test answers right away as the validation is also quite good. 

In [ ]:
#with test data
def refine(data):
    data = data.drop(['P8','V7','V9'],axis = 1)
    data = data.iloc[:,3:]
    data.iloc[:,:] = scaler.transform(data.iloc[:,:])
    return data

In [ ]:
test1 = refine(test1)

In [ ]:
test1

In [ ]:
test_result_tree = model.predict(test1)

In [ ]:
sum(test_result_tree==1)

In [ ]:
solution = pd.read_csv(data_dir+ '/solution.csv')

In [ ]:
solution

In [ ]:
target = solution['Prediction']

In [ ]:
accuracy_score(test_result_tree,target)

### Now the numbers are speaking. Lets have try at jungle model. You know it right!!!!

In [ ]:
jungle_solution = jungle.predict(test1)

In [ ]:
jungle_solution

In [ ]:
tologistic(jungle_solution)

In [ ]:
accuracy_score(jungle_solution, target)

### More accurate than the partner in crime.....But the real question is..
IS 77.78% enough??

In [ ]:
jovian.commit(project = 'my-project-001')

<IPython.core.display.Javascript object>

#### Time to fine tune the MODELS
R U REAADY????